### Problem 1: Nurse Scheduling with Preferences and Limits

You must assign nurses to shifts over a week.

- **Data**:
    - 7 days, 3 shifts/day (morning, afternoon, night)
    - 6 nurses
    - Each nurse can work at most 5 shifts a week and no more than one per day
    - No night shift followed by a morning shift the next day
    - Each shift must be covered by exactly one nurse
    - Nurses have preferences (0–10 score) for each shift
- **Objective**: Maximize total preference score across the week.

---

In [1]:
import gurobipy as gp
from gurobipy import GRB

def solve_nurse_scheduling(preferences):
    """
    Solves the nurse scheduling problem using Gurobipy.

    Args:
        preferences (dict): A dictionary where keys are (nurse, day, shift) tuples
                            and values are the preference scores (0-10).
                            Example: preferences[('N1', 'Mon', 'Morning')] = 8

    Returns:
        dict: A dictionary of assigned shifts if a solution is found,
              otherwise None.
    """

    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    shifts = ['Morning', 'Afternoon', 'Night']
    nurses = [f'N{i}' for i in range(1, 7)] # N1, N2, ..., N6 as Nurse Numbers

    max_shifts_per_week = 5
    max_shifts_per_day = 1

    model = gp.Model("Nurse_Scheduling")

    # --- Decision Variables ---
    # x[n, d, s] is 1 if nurse n works on day d, shift s; 0 otherwise
    x = model.addVars(nurses, days, shifts, vtype=GRB.BINARY, name="x")

    # --- Objective Function ---
    # Maximize total preference score
    model.setObjective(
        gp.quicksum(preferences.get((n, d, s), 0) * x[n, d, s]
                    for n in nurses for d in days for s in shifts),
        GRB.MAXIMIZE
    )

    # --- Constraints ---
    # 1. Each shift must be covered by exactly one nurse
    for d in days:
        for s in shifts:
            model.addConstr(
                gp.quicksum(x[n, d, s] for n in nurses) == 1,
                name=f"Shift_Coverage_{d}_{s}"
            )

    # 2. Each nurse can work at most max_shifts_per_week (5) shifts a week
    for n in nurses:
        model.addConstr(
            gp.quicksum(x[n, d, s] for d in days for s in shifts) <= max_shifts_per_week,
            name=f"Max_Shifts_Per_Week_{n}"
        )

    # 3. Each nurse can work at most max_shifts_per_day (1) shift per day
    for n in nurses:
        for d in days:
            model.addConstr(
                gp.quicksum(x[n, d, s] for s in shifts) <= max_shifts_per_day,
                name=f"Max_Shifts_Per_Day_{n}_{d}"
            )

    # 4. No night shift followed by a morning shift the next day
    for n in nurses:
        for i in range(len(days) - 1): # Iterate up to the second to last day
            today = days[i]
            next_day = days[i+1]
            model.addConstr(
                x[n, today, 'Night'] + x[n, next_day, 'Morning'] <= 1,
                name=f"No_Night_Morning_Overlap_{n}_{today}"
            )

    model.optimize()

    if model.status == GRB.OPTIMAL:
        print("\nOptimal solution found:")
        assigned_shifts = {}
        for n in nurses:
            for d in days:
                for s in shifts:
                    if x[n, d, s].x > 0.5: # Check if the variable is 1 (assigned)
                        assigned_shifts[(n, d, s)] = True
                        print(f"  Nurse {n} works on {d} - {s} shift")
        print(f"\nTotal Preference Score: {model.objVal}")
        return assigned_shifts
    elif model.status == GRB.INFEASIBLE:
        print("Model is infeasible.")
        return None
    elif model.status == GRB.UNBOUNDED:
        print("Model is unbounded.")
        return None
    else:
        print(f"Optimization ended with status {model.status}")
        return None

# --- Example Usage ---
if __name__ == "__main__":
    # Example Preference Data (replace with your actual data)
    # This is a random example to demonstrate the code.
    # In a real scenario, this would come from a database or input file.
    import random

    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    shifts = ['Morning', 'Afternoon', 'Night']
    nurses = [f'N{i}' for i in range(1, 7)]

    sample_preferences = {}
    for n in nurses:
        for d in days:
            for s in shifts:
                sample_preferences[(n, d, s)] = random.randint(0, 10) # Random preference for demonstration

    # You can also set specific preferences like this:
    # sample_preferences[('N1', 'Mon', 'Morning')] = 10
    # sample_preferences[('N2', 'Mon', 'Morning')] = 5

    scheduled_shifts = solve_nurse_scheduling(sample_preferences)

    if scheduled_shifts:
        # You can further process or display the scheduled_shifts dictionary
        pass

Set parameter Username
Set parameter LicenseID to value 2685582
Academic license - for non-commercial use only - expires 2026-07-08
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 105 rows, 126 columns and 450 nonzeros
Model fingerprint: 0xf532caf0
Variable types: 0 continuous, 126 integer (126 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 116.0000000
Presolve time: 0.00s
Presolved: 105 rows, 126 columns, 450 nonzeros
Variable types: 0 continuous, 126 integer (126 binary)

Root relaxation: objective 1.880000e+02, 58 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

---

### Problem 2: Production with Setup Times and Costs

A factory produces 4 products over 5 time periods.

- **Data**:
    - Each product requires different machine time and raw materials
    - Each switch to a new product has a fixed setup cost
    - Daily capacity for time and raw material
    - Storage cost for leftover products
    - Product demands per period
- **Objective**: Minimize total production + setup + storage costs.

---

In [10]:
import gurobipy as gp
from gurobipy import GRB

def solve_production_scheduling(
    products,
    periods,
    machine_time_req,
    raw_material_req,
    setup_cost,
    storage_cost,
    production_cost,
    daily_machine_capacity,
    daily_raw_material_capacity,
    demand,
    initial_inventory
):
    """
    Solves the production scheduling problem with setup costs and inventory.

    Args:
        products (list): List of product names (e.g., ['P1', 'P2']).
        periods (list): List of time period names (e.g., ['T1', 'T2']).
        machine_time_req (dict): Machine time required per unit for each product.
        raw_material_req (dict): Raw material required per unit for each product.
        setup_cost (dict): Fixed setup cost for each product.
        storage_cost (dict): Storage cost per unit per period for each product.
        production_cost (dict): Production cost per unit for each product.
        daily_machine_capacity (dict): Daily machine time capacity for each period.
        daily_raw_material_capacity (dict): Daily raw material capacity for each period.
        demand (dict): Demand for each product in each period.
        initial_inventory (dict): Initial inventory for each product.

    Returns:
        tuple: (optimal_cost, production_plan, inventory_plan, setup_plan) if optimal,
               otherwise (None, None, None, None).
    """

    model = gp.Model("Production_Scheduling")

    # --- Decision Variables ---
    # Produce[p, t]: Quantity of product p produced in period t
    Produce = model.addVars(products, periods, name="Produce")
    # Inventory[p, t]: Quantity of product p in inventory at the end of period t
    Inventory = model.addVars(products, periods, name="Inventory")
    # Setup[p, t]: 1 if product p has a setup in period t, 0 otherwise
    Setup = model.addVars(products, periods, vtype=GRB.BINARY, name="Setup")
    # IsProducing[p, t]: 1 if product p is produced in period t, 0 otherwise
    IsProducing = model.addVars(products, periods, vtype=GRB.BINARY, name="IsProducing")

    # --- Objective Function ---
    # Minimize total production + setup + storage costs
    model.setObjective(
        gp.quicksum(production_cost[p] * Produce[p, t] for p in products for t in periods) +
        gp.quicksum(setup_cost[p] * Setup[p, t] for p in products for t in periods) +
        gp.quicksum(storage_cost[p] * Inventory[p, t] for p in products for t in periods),
        GRB.MINIMIZE
    )

    # --- Constraints ---

    # 1. Inventory Balance
    for p in products:
        for i, t in enumerate(periods):
            # The demand for a product-period pair might not be in the dictionary.
            # .get() with a default value of 0 handles this safely.
            current_demand = demand.get((p, t), 0)
            if i == 0:  # First period
                model.addConstr(
                    initial_inventory.get(p, 0) + Produce[p, t] - current_demand == Inventory[p, t],
                    name=f"InvBalance_{p}_{t}"
                )
            else:  # Subsequent periods
                prev_t = periods[i-1]
                model.addConstr(
                    Inventory[p, prev_t] + Produce[p, t] - current_demand == Inventory[p, t],
                    name=f"InvBalance_{p}_{t}"
                )

    # 2. Machine Time Capacity
    for t in periods:
        model.addConstr(
            gp.quicksum(machine_time_req[p] * Produce[p, t] for p in products) <= daily_machine_capacity[t],
            name=f"MachineCapacity_{t}"
        )

    # 3. Raw Material Capacity
    for t in periods:
        model.addConstr(
            gp.quicksum(raw_material_req[p] * Produce[p, t] for p in products) <= daily_raw_material_capacity[t],
            name=f"RawMaterialCapacity_{t}"
        )

    # 4. Linking Produce_pt and IsProducing_pt (with dynamic Big-M)
    for p in products:
        # A safe upper bound for production of a product is its total demand
        # over the entire horizon, plus any initial inventory. You'd never
        # need to produce more than this amount in a single period.
        m_value = initial_inventory.get(p, 0) + sum(demand.get((p, t), 0) for t in periods)
        
        for t in periods:
            model.addConstr(
                Produce[p, t] <= m_value * IsProducing[p, t],
                name=f"LinkProduceIsProducing_{p}_{t}"
            )

    # 5. Setup Logic
    for p in products:
        for i, t in enumerate(periods):
            if i == 0: # Setup in the first period if produced
                model.addConstr(
                    Setup[p, t] >= IsProducing[p, t],
                    name=f"SetupLogic_Initial_{p}"
                )
            else: # Setup if produced in current period but not in previous
                prev_t = periods[i-1]
                model.addConstr(
                    Setup[p, t] >= IsProducing[p, t] - IsProducing[p, prev_t],
                    name=f"SetupLogic_{p}_{t}"
                )
    
    # --- Optimize the Model ---
    model.optimize()

    # --- Retrieve and Display Results ---
    if model.status == GRB.OPTIMAL:
        print("\n✅ Optimal solution found:")
        optimal_cost = model.objVal
        print(f"   Total Optimal Cost: ${optimal_cost:,.2f}")

        production_plan = {}
        inventory_plan = {}
        setup_plan = {}

        for p in products:
            for t in periods:
                production_plan[(p, t)] = Produce[p, t].x
                inventory_plan[(p, t)] = Inventory[p, t].x
                setup_plan[(p, t)] = Setup[p, t].x
        
        # Nicely print the plan
        for t in periods:
            print(f"\n--- Period: {t} ---")
            for p in products:
                prod_qty = production_plan.get((p, t), 0)
                inv_qty = inventory_plan.get((p, t), 0)
                is_setup = setup_plan.get((p, t), 0) > 0.5
                
                if prod_qty > 1e-6 or inv_qty > 1e-6:
                    setup_str = " (Setup Incurred)" if is_setup else ""
                    print(f"  Product {p}: Produce={prod_qty:<7.2f} | End Inventory={inv_qty:<7.2f}{setup_str}")

        return optimal_cost, production_plan, inventory_plan, setup_plan

    elif model.status == GRB.INFEASIBLE:
        print("\n❌ Model is infeasible.")
        model.computeIIS() # Compute Irreducible Inconsistent Subsystem
        iis_file = "production_infeasible.ilp"
        model.write(iis_file)
        print(f"   Infeasible constraints written to '{iis_file}'")
        return None, None, None, None
    elif model.status == GRB.UNBOUNDED:
        print("\n⚠️ Model is unbounded.")
        return None, None, None, None
    else:
        print(f"\nOptimization ended with status {model.status}")
        return None, None, None, None

# --- Example Usage ---
if __name__ == "__main__":
    # --- Sample Data (Replace with your actual data) ---
    products_data = ['P1', 'P2', 'P3', 'P4']
    periods_data = ['T1', 'T2', 'T3', 'T4', 'T5']

    machine_time_data = {'P1': 0.1, 'P2': 0.15, 'P3': 0.2, 'P4': 0.25}
    raw_material_data = {'P1': 0.05, 'P2': 0.08, 'P3': 0.1, 'P4': 0.12}
    setup_cost_data = {'P1': 500, 'P2': 700, 'P3': 600, 'P4': 800}
    storage_cost_data = {'P1': 2, 'P2': 3, 'P3': 2.5, 'P4': 3.5}
    production_cost_data = {'P1': 10, 'P2': 12, 'P3': 15, 'P4': 18}

    machine_capacity_data = {t: 480 for t in periods_data} 
    raw_material_capacity_data = {t: 300 for t in periods_data}

    demand_data = {
        ('P1', 'T1'): 50, ('P1', 'T2'): 60, ('P1', 'T3'): 70, ('P1', 'T4'): 80, ('P1', 'T5'): 90,
        ('P2', 'T1'): 40, ('P2', 'T2'): 50, ('P2', 'T3'): 60, ('P2', 'T4'): 70, ('P2', 'T5'): 80,
        ('P3', 'T1'): 30, ('P3', 'T2'): 40, ('P3', 'T3'): 50, ('P3', 'T4'): 60, ('P3', 'T5'): 70,
        ('P4', 'T1'): 20, ('P4', 'T2'): 30, ('P4', 'T3'): 40, ('P4', 'T4'): 50, ('P4', 'T5'): 60,
    }

    initial_inventory_data = {'P1': 10, 'P2': 5, 'P3': 0, 'P4': 0}

    # Solve the model with the provided data
    solve_production_scheduling(
        products=products_data,
        periods=periods_data,
        machine_time_req=machine_time_data,
        raw_material_req=raw_material_data,
        setup_cost=setup_cost_data,
        storage_cost=storage_cost_data,
        production_cost=production_cost_data,
        daily_machine_capacity=machine_capacity_data,
        daily_raw_material_capacity=raw_material_capacity_data,
        demand=demand_data,
        initial_inventory=initial_inventory_data
    )

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 70 rows, 80 columns and 192 nonzeros
Model fingerprint: 0x6d4519b8
Variable types: 40 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [5e-02, 4e+02]
  Objective range  [2e+00, 8e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 5e+02]
Found heuristic solution: objective 22450.000000
Presolve removed 66 rows and 72 columns
Presolve time: 0.00s
Presolved: 4 rows, 8 columns, 11 nonzeros
Found heuristic solution: objective 17170.000000
Variable types: 8 continuous, 0 integer (0 binary)

Root relaxation: objective 1.689000e+04, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0         

---

### Problem 3: Advertisement Budget Allocation (Piecewise Linear Approx.)

A company wants to allocate a monthly ad budget across 4 channels: social media, TV, newspaper, and online banners.

- **Data**:
    - Total budget = ₹1,00,000
    - Each channel has diminishing returns (e.g., modeled with piecewise linear utility functions)
    - Minimum and maximum spend per channel
    - TV must get at least 30% if chosen at all
    - At most 3 out of 4 channels can be active
- **Objective**: Maximize total reach.

---

In [12]:
import gurobipy as gp
from gurobipy import GRB

# --- 1. Define Data ---

# Channels and budget
channels = ['Social Media', 'TV', 'Newspaper', 'Banners']
total_budget = 100000

# Minimum and maximum spend per channel if active
# Note: The TV minimum is set to 30% of the total budget as per the rule.
min_spend = {
    'Social Media': 5000,
    'TV': 0.30 * total_budget,  # TV must get at least 30% of total budget if chosen
    'Newspaper': 3000,
    'Banners': 2000
}
max_spend = {
    'Social Media': 45000,
    'TV': 70000,
    'Newspaper': 25000,
    'Banners': 20000
}

# Piecewise linear data for reach (utility)
# For each channel, we define points (spend, reach) to model diminishing returns.
# Format: {channel: ([spend_points], [reach_points])}
reach_points = {
    'Social Media': ([0, 15000, 30000, 45000], [0, 120000, 190000, 220000]),
    'TV':           ([0, 30000, 50000, 70000], [0, 300000, 450000, 500000]),
    'Newspaper':    ([0, 10000, 20000, 25000], [0, 80000,  130000, 150000]),
    'Banners':      ([0, 5000,  10000, 20000], [0, 40000,  60000,  70000])
}

# --- 2. Create the Gurobi Model ---
model = gp.Model("AdBudgetAllocation")

# --- 3. Define Decision Variables ---

# `spend[c]` is the continuous variable for the amount of money spent on channel c.
spend = model.addVars(channels, name="spend", lb=0)

# `use[c]` is a binary variable, 1 if channel c is used, 0 otherwise.
use = model.addVars(channels, vtype=GRB.BINARY, name="use")


# --- 4. Define the Objective Function ---

# We want to maximize the total reach. Gurobi's setPWLObj function
# is used here to define the objective contribution from each 'spend' variable
# according to the piecewise linear data points.
# This is a powerful feature that simplifies the model significantly.
for c in channels:
    x_pts, y_pts = reach_points[c]
    model.setPWLObj(spend[c], x_pts, y_pts)

# Set the model sense to maximization
model.ModelSense = GRB.MAXIMIZE


# --- 5. Add Constraints ---

# Constraint 1: Total budget constraint
model.addConstr(gp.quicksum(spend[c] for c in channels) <= total_budget, name="TotalBudget")

# Constraint 2: At most 3 out of 4 channels can be active
model.addConstr(gp.quicksum(use[c] for c in channels) <= 3, name="MaxChannels")

# Constraint 3: Link spend and use variables (min/max spend)
# If a channel is used (use[c] = 1), its spend must be between min and max.
# If a channel is not used (use[c] = 0), its spend must be 0.
for c in channels:
    model.addConstr(spend[c] >= min_spend[c] * use[c], name=f"MinSpend_{c}")
    model.addConstr(spend[c] <= max_spend[c] * use[c], name=f"MaxSpend_{c}")


# --- 6. Optimize the Model ---
print("Optimizing ad budget allocation...")
model.optimize()
print("------------------------------------")


# --- 7. Display the Results ---
if model.status == GRB.OPTIMAL:
    print(f"Optimal solution found!")
    print(f"   Total maximized reach: {model.objVal:,.0f} people")
    print("\n--- Optimal Budget Allocation ---")
    total_spent = 0
    for c in channels:
        if use[c].X > 0.5: # If the channel is used
            amount_spent = spend[c].X
            total_spent += amount_spent
            print(f"  - {c:<15}: ₹ {amount_spent:9,.2f}")

    print("------------------------------------")
    print(f"Total budget spent: ₹ {total_spent:9,.2f}")
    print(f"Budget remaining:   ₹ {(total_budget - total_spent):9,.2f}")

elif model.status == GRB.INFEASIBLE:
    print("Model is infeasible. Check constraints and data.")

else:
    print(f"Optimization finished with status: {model.status}")

Optimizing ad budget allocation...
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 10 rows, 8 columns and 24 nonzeros
Model fingerprint: 0x397b0553
Model has 4 piecewise-linear objective terms
Variable types: 4 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+04]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
  PWLObj x range   [5e+03, 5e+04]
  PWLObj obj range [4e+04, 4e+05]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 14 rows, 20 columns, 40 nonzeros
Variable types: 16 continuous, 4 integer (4 binary)

Root relaxation: objective 7.866667e+05, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

---

## Final Project: 3D Packing Optimization for Air Cargo (FedEx Case)

### Problem Statement

You are tasked with optimizing the **loading of packages into Unit Load Devices (ULDs)** - large containers used to efficiently transport cargo in aircraft. Your goal is to design a mathematical model that can **pack a set of 3D boxes into a fixed-size ULD** such that:

- All items are **non-overlapping** and **fully contained** within the ULD bounds.
- Constraints related to **weight balance**, **orientation**, and **priority items** (if introduced) are respected.
- The objective is to either:
    - **Maximize the number of items packed**, or
    - **Maximize the total packed volume/weight**, depending on the variation.

This mimics real-world logistics problems faced by FedEx and other cargo companies, where space and weight utilization directly impact cost and efficiency.

---

In [17]:
import gurobipy as gp
from gurobipy import GRB
import itertools

# --- 1. Define Input Data ---

# ULD (container) dimensions
ULD_DIMS = (100, 100, 100)  # Length, Width, Height

# Package data: {ID: {'dims': (l, w, h), 'priority': bool}}
BOXES = {
    1: {'dims': (40, 40, 40), 'priority': True},
    2: {'dims': (50, 50, 20), 'priority': False},
    3: {'dims': (30, 30, 30), 'priority': False},
    4: {'dims': (20, 60, 20), 'priority': False},
    5: {'dims': (70, 30, 40), 'priority': True},
}

box_ids = list(BOXES.keys())
N = len(box_ids)

# --- 2. Pre-process Rotations ---

# Generate all 6 unique, axis-aligned orientations for each box
box_orientations = {}
for i in box_ids:
    # Use a set to store unique dimension permutations
    unique_dims = set(itertools.permutations(BOXES[i]['dims']))
    box_orientations[i] = [list(perm) for perm in unique_dims]

# --- 3. Create the Gurobi Model ---

model = gp.Model("3D_Packing_Optimization")

# --- 4. Define Decision Variables ---

# pack[i]: Binary variable, 1 if box i is packed, 0 otherwise
pack = model.addVars(box_ids, vtype=GRB.BINARY, name="pack")

# pos[i]: Continuous variables for the front-lower-left corner (x,y,z) of box i
pos = {i: model.addVars(3, name=f"pos_{i}", lb=0) for i in box_ids}

# rot[i][j]: Binary variable, 1 if box i uses orientation j
rot = {}
for i in box_ids:
    rot[i] = model.addVars(len(box_orientations[i]), vtype=GRB.BINARY, name=f"rot_{i}")

# Effective dimensions of each box after rotation
eff_dims = {}
for i in box_ids:
    eff_dims[i] = [
        gp.quicksum(rot[i][j] * box_orientations[i][j][k] for j in range(len(box_orientations[i])))
        for k in range(3)
    ]

# Relative position binaries: 1 if box i is to the left of/behind/below box j
# This is the core of the non-overlapping constraints
rel_pos = model.addVars(box_ids, box_ids, 3, vtype=GRB.BINARY, name="rel_pos")


# --- 5. Define Constraints ---

# For every pair of distinct boxes (i, j)
for i, j in itertools.combinations(box_ids, 2):
    # Non-overlapping constraint:
    # At least one of the 6 relative positioning conditions must hold
    # This is only enforced if BOTH boxes i and j are packed.
    model.addConstr(
        rel_pos[i, j, 0] + rel_pos[j, i, 0] +  # i is left of j OR j is left of i
        rel_pos[i, j, 1] + rel_pos[j, i, 1] +  # i is behind j OR j is behind i
        rel_pos[i, j, 2] + rel_pos[j, i, 2]    # i is below j OR j is below i
        >= pack[i] + pack[j] - 1
    )

    # Link position variables to the relative position binaries (Big-M method)
    # M is a large number; the ULD dimension is a safe choice.
    for k in range(3): # For each axis (x, y, z)
        # i is left of/behind/below j
        model.addConstr(pos[i][k] + eff_dims[i][k] <= pos[j][k] + ULD_DIMS[k] * (1 - rel_pos[i, j, k]))
        # j is left of/behind/below i
        model.addConstr(pos[j][k] + eff_dims[j][k] <= pos[i][k] + ULD_DIMS[k] * (1 - rel_pos[j, i, k]))

for i in box_ids:
    # Rotation constraint: If a box is packed, exactly one orientation must be chosen
    model.addConstr(gp.quicksum(rot[i]) == pack[i])

    # Boundary constraint: If a box is packed, it must be fully within the ULD
    for k in range(3):
        # The position variables are only meaningful if the box is packed.
        # This Big-M constraint ensures coordinates are forced to 0 if not packed,
        # and the box fits within the ULD if it is packed.
        model.addConstr(pos[i][k] + eff_dims[i][k] <= ULD_DIMS[k] + ULD_DIMS[k] * (1 - pack[i]))

# --- 6. Define Objective Function ---

# Maximize the number of boxes packed, with a heavy preference for priority items.
# A priority item is given a weight of N+1 to ensure it's always chosen over
# any number of non-priority items.
priority_weight = N + 1
model.setObjective(
    gp.quicksum(priority_weight * pack[i] for i in box_ids if BOXES[i]['priority']) +
    gp.quicksum(pack[i] for i in box_ids if not BOXES[i]['priority']),
    GRB.MAXIMIZE
)


# --- 7. Optimize the Model ---

print("Starting 3D packing optimization...")
model.optimize()


# --- 8. Display the Results ---
if model.status == GRB.OPTIMAL:
    print("\nOptimal packing solution found!")
    
    packed_count = int(sum(pack[i].X for i in box_ids))
    print(f"\nTotal Boxes Packed: {packed_count} out of {N}")

    print("\n--- Packed Box Details ---")
    print(f"{'ID':<5} {'Packed':<8} {'Position (x, y, z)':<25} {'Orientation (L×W×H)':<25}")
    print("-" * 80)
    
    for i in box_ids:
        if pack[i].X > 0.5: # If the box is packed
            
            # Find the chosen orientation
            chosen_orientation = None
            for j in range(len(box_orientations[i])):
                if rot[i][j].X > 0.5:
                    chosen_orientation = box_orientations[i][j]
                    break
            
            position_str = f"({pos[i][0].X:.0f}, {pos[i][1].X:.0f}, {pos[i][2].X:.0f})"
            orientation_str = f"{chosen_orientation[0]} × {chosen_orientation[1]} × {chosen_orientation[2]}"
            
            print(f"{i:<5} {'Yes':<8} {position_str:<25} {orientation_str:<25}")
        else:
            print(f"{i:<5} {'No':<8} {'-':<25} {'-':<25}")

else:
    print(f"\nNo optimal solution found. Status code: {model.status}")
    if model.status == GRB.INFEASIBLE:
        print("   Model is infeasible. Check for conflicting constraints.")
        # Compute and write an Irreducible Inconsistent Subsystem (IIS) to a file
        model.computeIIS()
        model.write("packing_infeasible_model.ilp")
        print("   IIS written to 'packing_infeasible_model.ilp' for debugging.")

Starting 3D packing optimization...
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 90 rows, 109 columns and 519 nonzeros
Model fingerprint: 0x68ae4a62
Variable types: 15 continuous, 94 integer (94 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 2 rows and 17 columns
Presolve time: 0.00s
Presolved: 88 rows, 92 columns, 494 nonzeros
Variable types: 15 continuous, 77 integer (77 binary)
Found heuristic solution: objective 6.0000000

Root relaxation: objective 1.500000e+01, 23 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap

/var/folders/yh/r_y0_1096fx80kg5yrpqly5r0000gn/T/ipykernel_45845/3260633355.py:85: DeprecationWarning: Calling quicksum on a tupledict is deprecated, use .sum() instead.
  model.addConstr(gp.quicksum(rot[i]) == pack[i])
